In [1]:
import pandas as pd
import numpy as np
import re
from papyrus_scripts.download import download_papyrus


In [2]:
download_papyrus(version='latest', structures=True, descriptors=['mold2', 'unirep'])

Latest version: 05.5
Number of files to be donwloaded: 9
Total size: 2.80GB


Donwloading version 05.5:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

In [3]:
from papyrus_scripts.reader import read_papyrus, read_protein_set

In [4]:
sample_data = read_papyrus(is3d=False, chunksize=1000000, source_path=None)

In [5]:
#chunk1 = next(sample_data)
#chunk1

In [6]:
#pd.set_option('display.max_columns', None) # to show all columns in the dataframe
#chunk1.head()

In [7]:
protein_data = read_protein_set(source_path=None)
protein_data.head()

,target_id,HGNC_symbol,UniProtID,Status,Organism,Classification,Length,Sequence
0,P47747_WT,,HRH2_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,359,MAFNGTVPSFCMDFTVYKVTISVILIILILVTVAGNVVVCLAVGLN...
1,B0FL73_WT,,B0FL73_CAVPO,unreviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,467,MGAGVLALGASEPCNLSSTAPLPDGAATAARLLVPASPPASLLPPT...
2,Q8K4Z4_WT,,ADRB2_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,418,MGHLGNGSDFLLAPNASHAPDHNVTRERDEAWVVGMAIVMSLIVLA...
3,P97266_WT,,OPRM_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,98,YTKMKTATNIYIFNLALADALATSTLPFQSVNYLMGTWPFGTILCK...
4,P41144_WT,,OPRK_CAVPO,reviewed,Cavia porcellus (Guinea pig),Membrane receptor->Family A G protein-coupled ...,380,MGRRRQGPAQPASELPARNACLLPNGSAWLPGWAEPDGNGSAGPQD...


In [8]:
from papyrus_scripts.preprocess import (keep_quality, keep_source, keep_type,
                                        keep_organism, keep_accession, keep_protein_class,
                                        keep_match, keep_contains
                                       )

The next part was used to make sure the filtering step works on a smaller sample of the data prior to filtering the whole dataset. 

In [9]:
#filter1 = keep_accession(data=chunk1, accession=["P51681", "P41597"])
#filter1.head()


In [10]:
#print(f'Number of activity points: {filter1.shape[0]}')

From this point forward the entire dataset will be filtered

In [11]:
filter1_it = keep_accession(data=sample_data, accession=["P41597"]) #CCR2: P41597 ; CCR5: P51681

In [12]:
#filter1_it

In [13]:
from papyrus_scripts.preprocess import consume_chunks

In [14]:
filtered_data_CCR2 = consume_chunks(filter1_it, progress=True, total=60)

  0%|          | 0/60 [00:00<?, ?it/s]

In [15]:
print(f'Number of activity points: {filtered_data_CCR2.shape[0]}')

Number of activity points: 1688


In [16]:
data_drop_CCR2 = filtered_data_CCR2.drop_duplicates(subset=['InChIKey'])
data_drop_CCR2.count()

Activity_ID             1688
Quality                 1688
source                  1688
CID                     1688
SMILES                  1688
connectivity            1688
InChIKey                1688
InChI                   1688
InChI_AuxInfo           1688
target_id               1688
accession               1688
Protein_Type            1688
AID                     1688
doc_id                  1682
Year                    1682
all_doc_ids             1632
all_years               1632
type_IC50               1688
type_EC50               1688
type_KD                 1688
type_Ki                 1688
type_other              1688
Activity_class             2
relation                1686
pchembl_value           1686
pchembl_value_Mean      1686
pchembl_value_StdDev    1686
pchembl_value_SEM       1686
pchembl_value_N         1686
pchembl_value_Median    1686
pchembl_value_MAD       1686
dtype: int64

In [32]:
#export DataFrame to CSV file
data_drop_CCR2.to_csv(r'C:\documenten hier\ccr2_data.csv', index=False)

In [18]:
filter2_it = keep_accession(data=sample_data, accession=["P51681"]) #CCR5: P51681

In [19]:
filtered_data_CCR5 = consume_chunks(filter2_it, progress=True, total=60)

In [20]:
print(f'Number of activity points: {filtered_data_CCR5.shape[0]}')

In [21]:
data_drop_CCR5 = filtered_data_CCR5.drop_duplicates(subset=['InChIKey'])

In [22]:
data_drop_CCR5.count()

In [23]:
#export DataFrame to CSV file
data_drop_CCR5.to_csv(r'C:\documenten hier\ccr5_data.csv', index=False)

In [24]:
filtered_data['accession'].value_counts().to_frame()

In [25]:
filtered_data['target_id'].value_counts().to_frame()

In [26]:
import matplotlib.pyplot as plt
df_2 = pd.DataFrame(filtered_data.groupby(['accession'])['accession'].count()).rename({'accession':'count'},axis=1)
f_2 = df_2.sort_values(by='count',ascending=False)
y_pos = np.arange(len(df_2))
plt.bar(y_pos,df_2['count'], align='center', alpha=0.5)
plt.xticks(y_pos, df_2.index)
plt.ylabel('Frequency')
plt.title('accession')
plt.show()